# TFM - Xavier Velázquez Redondo

En este documento se muestra el desarrollo del modelo de aprendizaje para la predicción de los costes de generación en las islas canarias. Se utiliza lenguaje Python por su versatilidad y gran cantidad de librerías.

En primer lugar importamos las librerías que nos serán de utilidad:

In [57]:
# import requests library
import requests 
import json
# import pandas library
import pandas as pd 
# import csv library
import csv
# import datetime library
import datetime

As we have seen before, when we work with APIs, we have to write the endpoint we want to access. 
We need to write also the folder/section we want to access. We can have the same endpoint and different sections from where to get information. 

Most of the times, we need to add some headers to our requests that are required to get the information. The most important one is the **Authorization**, which corresponds to our ID to access that API.

## Listado de indicadores

Podemos llamar a objetos a traves de la API. Esto lo hacemos mediante la librería requests que nos permite hacer solicitudes HTTP a un servidor mediante el método GET de entre otros.

A continuación hacemos una solicitud del listado de indicadores existentes a través de la API, comprobamos su estado y imprimimos en pantalla los valores para su comprensión.

In [58]:
endpoint = 'https://api.esios.ree.es'
get_indicators = '/indicators?text=SNP'
headers = {'Accept': 'application/json;application/vnd.esios-api-v1+json',
           'Host': 'api.esios.ree.es',
           'Authorization': 'Token token=\"3627f9a1b46617055cb615ed161382e551ec7baa18466965dfd2cda3ebfc7307"',
          }

request = requests.get(endpoint+get_indicators, headers=headers) #, params=params)
status = request.status_code

#Check the status code
if status < 200:
    print('informational')
    # If the status code is 200, treat the information.
elif status >= 200 and status < 300:
    print('Connection is established')
    #okBehavior(response) # runs the function to get list of archives
elif status >= 300 and status < 400:
    print('redirection')
elif status >= 400 and status < 500:
    print('client error')
else:
    print('server error')

request.json()

Connection is established


{'indicators': [{'name': 'Generación T.Real enlace Ibiza-Formentera SNP',
   'description': '<p>Generaci&oacute;n medida en tiempo real en el enlace Ibiza-Formentera&nbsp;en el sistema Balear.</p>\n\n<p><b>Publicaci&oacute;n:</b>&nbsp;cada 10 minutos con la informaci&oacute;n de las tres &uacute;ltimas horas del d&iacute;a D-1 hasta la hora actual del d&iacute;a D.</p>\n',
   'short_name': 'Enlace Ibiza-Formentera SNP',
   'id': 1820},
  {'name': 'Generación T.Real enlace Mallorca-Ibiza SNP',
   'description': '<p>Generaci&oacute;n medida en tiempo real en el enlace Mallorca-Ibiza&nbsp;en el sistema Balear.</p>\n\n<p><b>Publicaci&oacute;n:</b>&nbsp;cada 10 minutos con la informaci&oacute;n de las tres &uacute;ltimas horas del d&iacute;a D-1 hasta la hora actual del d&iacute;a D.</p>\n',
   'short_name': 'Enlace Mallorca-Ibiza SNP',
   'id': 1822},
  {'name': 'Generación T.Real enlace Mallorca-Menorca SNP',
   'description': '<p>Generaci&oacute;n medida en tiempo real en el enlace Mallo

#### Indicadores de interés:

ID:10354 Generación T.Real no renovable SNP (desglose subsistemas, granularidad 10')
ID:10353 Generación T.Real renovable SNP (desglose subsistemas, granularidad 10')
ID:10350 Demanda real suma de generación SNP (desglose subsistemas, granularidad 10')
ID:1755 Generación T.Real cogeneración SNP (desglose subsistemas, granularidad 10')
ID:1754 Generación T.Real generación auxiliar SNP (desglose subsistemas, granularidad 10')
ID:1753 Generación T.Real residuos SNP (desglose subsistemas, granularidad 10')
ID:1752 Generación T.Real otras renovables SNP (desglose subsistemas, granularidad 10')
ID:1750 Generación T.Real carbón SNP (desglose subsistemas, granularidad 10')
ID:1749 Generación T.Real hidráulica SNP (desglose subsistemas, granularidad 10')
ID:1748 Generación T.Real solar fotovoltaica SNP (desglose subsistemas, granularidad 10')
ID:1747 Generación T.Real turbina de vapor SNP (desglose subsistemas, granularidad 10')
ID:1746 Generación T.Real ciclo combinado SNP (desglose subsistemas, granularidad 10')
ID:1745 Generación T.Real eólica SNP (desglose subsistemas, granularidad 10')
ID:1744 Generación T.Real turbina de gas SNP (desglose subsistemas, granularidad 10')
ID:1743 Generación T.Real motores diésel SNP (desglose subsistemas, granularidad 10')
ID:1751 Generación T.Real enlace balear SNP (desglose subsistemas, granularidad 10')
ID:1742 Demanda prevista SNP (desglose subsistemas, granularidad 10')
ID:1741 Demanda programada SNP (desglose subsistemas, granularidad 10')
ID:1740 Demanda Real SNP (desglose subsistemas, granularidad 10')


ID:10248 Generación en los SNP (desglose subsistemas, granularidad horaria)
ID:10246 Generación en Canarias (desglose subsistemas, granularidad horaria)
ID:10245 Demanda en los SNP (desglose subsistemas, granularidad horaria)
ID:10243 Demanda en Canarias (desglose subsistemas, granularidad horaria)
ID:1337 Precio medio de la generación en los SNP por subsistema (desglose subsistemas, granularidad horaria)
ID:1336 Precio medio de la demanda en los SNP por subsistema (desglose subsistemas, granularidad horaria)


ID:573 Precio medio de la demanda en los SNP por sistema (desglose sistemas, granularidad horaria)
ID:574 Precio medio de la generación en los SNP por sistema (desglose sistemas, granularidad horaria)

## Indicador X (pruebas)

In [59]:
date_start = "2019-12-31"                # start date
date_end = "2019-12-31"                  # end date

startdate = date_start + "T00:00:00.00" # can specify more if desired
enddate = date_end + "T23:50:00.00"
indicator = str(574)    # INDICADOR

#Energía correspondiente al programa operativo que el OS establece en cada período hasta el final del horizonte de programación diario

# You can get it by putting the mouse over the indicator name on the webpage. 
# webpage: https://www.esios.ree.es/es/analisis/10024
website = 'https://api.esios.ree.es/indicators/'+indicator+'?start_date='+startdate+'&end_date='+enddate#+'&geo_id[]='+geo_ids
     
     
print('Checking dates: ' , startdate , "to" , enddate) # printing the date 

URL = website # host website
GET = '/archives_json' # API link
HEADERS = {
            'Accept': "application/json; application/vnd.esios-api-v1+json",
            'Host': 'api.esios.ree.es',
            'Authorization': "Token token=\"3627f9a1b46617055cb615ed161382e551ec7baa18466965dfd2cda3ebfc7307",#"a6f2f926dea90ade64acc97b3b4fff73af5bdc5d7bce554a1adfa16d554ede81"',
            'Content-Type': 'application/json'}
#PARAMS = {'date':date}

# Runs the request to get the total URL with access token
response = requests.get(url = URL+GET, headers = HEADERS)#, params = PARAMS)

# Read the status code
status = response.status_code  

#print the json()
response.json()

Checking dates:  2019-12-31T00:00:00.00 to 2019-12-31T23:50:00.00


{'indicator': {'name': 'Precio medio de la generación en los SNP por sistema',
  'short_name': 'Precio medio generación sistema',
  'id': 574,
  'composited': False,
  'step_type': 'linear',
  'disaggregated': True,
  'magnitud': [{'name': 'Precio', 'id': 23}],
  'tiempo': [{'name': 'Hora', 'id': 4}],
  'geos': [{'geo_id': 8742, 'geo_name': 'Canarias'},
   {'geo_id': 8743, 'geo_name': 'Baleares'},
   {'geo_id': 8744, 'geo_name': 'Ceuta'},
   {'geo_id': 8745, 'geo_name': 'Melilla'}],
  'values_updated_at': '2020-08-28T08:11:12.000+02:00',
  'values': [{'value': 138.74,
    'datetime': '2019-12-31T00:00:00.000+01:00',
    'datetime_utc': '2019-12-30T23:00:00Z',
    'tz_time': '2019-12-30T23:00:00.000Z',
    'geo_id': 8742,
    'geo_name': 'Canarias'},
   {'value': 94.65,
    'datetime': '2019-12-31T00:00:00.000+01:00',
    'datetime_utc': '2019-12-30T23:00:00Z',
    'tz_time': '2019-12-30T23:00:00.000Z',
    'geo_id': 8743,
    'geo_name': 'Baleares'},
   {'value': 185.25,
    'datetime'

In [69]:
date_start = "2019-12-31"                # start date
date_end = "2019-12-31"                  # end date

startdate = date_start + "T00:00:00.00" # can specify more if desired
enddate = date_end + "T23:50:00.00"

#LISTADO DE INDICADORES:

#ID:10354 Generación T.Real no renovable SNP (desglose subsistemas, granularidad 10')
#ID:10353 Generación T.Real renovable SNP (desglose subsistemas, granularidad 10')
#ID:10350 Demanda real suma de generación SNP (desglose subsistemas, granularidad 10')
#ID:1749 Generación T.Real hidráulica SNP (desglose subsistemas, granularidad 10')
#ID:1748 Generación T.Real solar fotovoltaica SNP (desglose subsistemas, granularidad 10')
#ID:1747 Generación T.Real turbina de vapor SNP (desglose subsistemas, granularidad 10')
#ID:1746 Generación T.Real ciclo combinado SNP (desglose subsistemas, granularidad 10')
#ID:1745 Generación T.Real eólica SNP (desglose subsistemas, granularidad 10')
#ID:1744 Generación T.Real turbina de gas SNP (desglose subsistemas, granularidad 10')
#ID:1743 Generación T.Real motores diésel SNP (desglose subsistemas, granularidad 10')
#ID:1742 Demanda prevista SNP (desglose subsistemas, granularidad 10')
#ID:1741 Demanda programada SNP (desglose subsistemas, granularidad 10')
#ID:1740 Demanda Real SNP (desglose subsistemas, granularidad 10')

#ID:10248 Generación en los SNP (desglose subsistemas, granularidad horaria)
#ID:10246 Generación en Canarias (desglose subsistemas, granularidad horaria) IGUAL QUE EL DE ARRIBA
#ID:10245 Demanda en los SNP (desglose subsistemas, granularidad horaria)
#ID:10243 Demanda en Canarias (desglose subsistemas, granularidad horaria) IGUAL QUE EL DE ARRIBA
#ID:1337 Precio medio de la generación en los SNP por subsistema (desglose subsistemas, granularidad horaria)
#ID:1336 Precio medio de la demanda en los SNP por subsistema (desglose subsistemas, granularidad horaria)

#ID:573 Precio medio de la demanda en los SNP por sistema (desglose sistemas, granularidad horaria)
#ID:574 Precio medio de la generación en los SNP por sistema (desglose sistemas, granularidad horaria)

datos_id = ["10350","1749","1748","1747","1746","1745","1744","1743","1741","1740","1337","1336"]
dataset=pd.DataFrame(columns=('ini','time_series'))

#LISTADO DE IDENTIFICADORES GEOGRÁFICOS:

#ID:8795 Gran Canaria
#ID:8796 Lanzarota y Fuerteventura
#ID:8797 Tenerife
#ID:8798 La Palma
#ID:8799 La Gomera
#ID:8800 El Hierro

#ID:8742 Islas Canarias


# You can get it by putting the mouse over the indicator name on the webpage. 
# webpage: https://www.esios.ree.es/es/analisis/10354

print ("Recorrer lista por Indices")
for x in range(0,len(datos_id)):
    website = 'https://api.esios.ree.es/indicators/'+datos_id[x]+'?start_date='+startdate+'&end_date='+enddate
    
    URL = website # host website
    GET = '/archives_json' # API link
    HEADERS = {
                'Accept': "application/json; application/vnd.esios-api-v1+json",
                'Host': 'api.esios.ree.es',
                'Authorization': "Token token=\"3627f9a1b46617055cb615ed161382e551ec7baa18466965dfd2cda3ebfc7307",#"a6f2f926dea90ade64acc97b3b4fff73af5bdc5d7bce554a1adfa16d554ede81"',
                'Content-Type': 'application/json'}
    
    # Runs the request to get the total URL with access token
    response = requests.get(url = URL+GET, headers = HEADERS)#, params = PARAMS) 
    matriz=[]
       
    for stuff in response.json()["indicator"]['values']:
        #print(stuff) ## show more stuff that can be gathered
        if stuff['geo_id'] == 8797: ## choose Tenerife id=8797 
            #print(stuff['value'])
            matriz.append([pd.to_datetime(stuff['datetime'],format='%Y-%m-%dT%H:%M:%S.%f+01:00'),stuff['value']])
            encabezado = ["time_series",response.json()["indicator"]['name']]
         
    df_parcial=pd.DataFrame(data=matriz,columns=encabezado)
    dataset=pd.merge(dataset,df_parcial, how='outer', on='time_series')
    
                 
dataset = dataset.set_index('time_series')
#dataset = dataset.resample('60T').mean()

#x=dataset.groupby(dataset.index.hour)['Precio medio de la demanda en los SNP por subsistema'].mean()
#print(x)
print('Checking dates: ' , startdate , "to" , enddate) # printing the date 


# Read the status code
status = response.status_code

print(dataset)

#dataset.to_csv('df_Tenerife_Electricity.csv', index=True)


Recorrer lista por Indices
Checking dates:  2019-12-31T00:00:00.00 to 2019-12-31T23:50:00.00
                     ini  Demanda real suma de generación SNP  \
time_series                                                     
2019-12-31 00:00:00  NaN                                356.4   
2019-12-31 00:10:00  NaN                                352.0   
2019-12-31 00:20:00  NaN                                345.5   
2019-12-31 00:30:00  NaN                                340.5   
2019-12-31 00:40:00  NaN                                332.5   
...                  ...                                  ...   
2019-12-31 23:10:00  NaN                                373.7   
2019-12-31 23:20:00  NaN                                370.4   
2019-12-31 23:30:00  NaN                                365.3   
2019-12-31 23:40:00  NaN                                360.2   
2019-12-31 23:50:00  NaN                                355.9   

                     Generación T.Real hidráulica SNP  \
time